In [1]:
import pandas as pd
from notebooks.extract import extract_csv
etapa = "chamada_regular"
# etapa = "lista_de_espera"
ano = 2017
semestre = 2
arquivo_nome = f"{etapa}_sisu_{ano}_{semestre}"
arquivo_path = f'./sisu/{ano}/{arquivo_nome}.csv'
transformado_path = f'./transformado/{ano}/{arquivo_nome}.csv'
nome_tabela = arquivo_nome.replace('sisu_', '').replace('de_', '')
print(nome_tabela, "PATH ALVO: ",transformado_path)


chamada_regular_2017_2 PATH ALVO:  ./transformado/2017/chamada_regular_sisu_2017_2.csv


[Dicionário de dados](https://dadosabertos.mec.gov.br/sisu/item/133-dicionario-de-dados)
para dataframes recentes.


In [2]:
# dataframe de referencia, leve e com colunas padronizadas:
df_ref = pd.read_csv(
    "./reduzido/2022/reduzido_lista_de_espera_sisu_2022_2.csv",
    sep="|",
    encoding="ISO-8859-1",
)


In [3]:
df_alvo = extract_csv(ano, etapa, semestre)
df_alvo.shape
# df_alvo = pd.read_csv(
#     f"./sisu/{ano}/{etapa}_sisu_{ano}_{semestre}.csv", sep=";", encoding="utf-8"
# )  # sep diferente

/home/vrsm/projetos/sisu1722/notebooks/extract.py:14: DtypeWarning: Columns (19,20,21,22,23,47) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(


(1797386, 53)

In [4]:
# df_alvo = pd.read_csv(
#     "./sisu/2022/chamada_regular_sisu_2022_2.csv", sep="|", encoding="ISO-8859-1"
# )  # sep diferente
# df_alvo.shape

In [5]:
# renomeia colunas
new_columns = {col: col.replace("NU_", "") for col in df_alvo.columns if "NU_" in col}
sg__columns = {col: col.replace("SG_", "") for col in df_alvo.columns if "SG_" in col}
st__columns = {col: col.replace("ST_", "") for col in df_alvo.columns if "ST_" in col}
no__columns = {
    col: col.replace("NO_", "NOME_") for col in df_alvo.columns if "NO_" in col
}
ind_columns = {
    "DT_NASCIMENTO": "DATA_NASCIMENTO",
    "TP_SEXO": "SEXO",
    "TP_MOD_CONCORRENCIA": "TIPO_MOD_CONCORRENCIA",
    "CO_ETAPA": "ETAPA",
    "CAMPUS": "NOME_CAMPUS",
    "CO_CAMPUS": "CODIGO_CAMPUS",
    "CO_IES": "CODIGO_IES",
    "IES": "NOME_IES",
    "CURSO": "NOME_CURSO",
    "CO_IES_CURSO": "CODIGO_CURSO",
    "CO_INSCRICAO_ENEM": "INSCRICAO_ENEM",
    "DS_GRAU": "GRAU",
    "DS_TURNO": "TURNO",
    "DS_MOD_CONCORRENCIA": "MOD_CONCORRENCIA",
}

l_cols = {
    "NOTACORTE": "NOTA_CORTE",
    "NOME_MUNUCIPIO_CAMPUS": "MUNICIPIO_CAMPUS",
    "NOME_INSCRITO": "INSCRITO",
}

new_columns.update(sg__columns)
new_columns.update(st__columns)
new_columns.update(no__columns)
new_columns.update(ind_columns)

df_alvo = df_alvo.rename(columns=new_columns).copy()
df_alvo = df_alvo.rename(columns=l_cols).copy()
df_alvo['CLASSIFICACAO'] = df_alvo['CLASSIFICACAO'].fillna(0).astype(float).astype(int)

In [6]:
try:
    df_ref = df_ref.copy().drop(
        ["DS_PERIODICIDADE", "QT_VAGAS_CONCORRENCIA", "TP_COTA", "PERCENTUAL_BONUS"],
        axis=1,
    )
except KeyError:
    print("colunas não encontradas")
    
try:
    df_alvo = df_alvo.copy().drop(["PERCENTUAL_BONUS"], axis=1)
except KeyError:
    print("colunas não encontradas")
    
try:
    df_alvo = df_alvo.copy().drop(
        ["DS_PERIODICIDADE", "QT_VAGAS_CONCORRENCIA", "TP_COTA"], axis=1
    )
except KeyError:
    print("colunas não encontradas")

print(df_alvo.columns)
print(df_ref.columns)
# colunas diferentes
print("shape df_alvo:", df_alvo.shape)  # 53
print("shape df_ref:", df_ref.shape)  # 56
# colunas a mais: DS_PERIODICIDADE, TP_COTA, QT_VAGAS_CONCORRENCIA

colunas não encontradas
Index(['ANO', 'EDICAO', 'ETAPA', 'DS_ETAPA', 'CODIGO_IES', 'NOME_IES',
       'SIGLA_IES', 'UF_IES', 'CODIGO_CAMPUS', 'NOME_CAMPUS', 'UF_CAMPUS',
       'MUNICIPIO_CAMPUS', 'CODIGO_CURSO', 'NOME_CURSO', 'GRAU', 'TURNO',
       'TIPO_MOD_CONCORRENCIA', 'MOD_CONCORRENCIA', 'PESO_L', 'PESO_CH',
       'PESO_CN', 'PESO_M', 'PESO_R', 'NOTA_MINIMA_L', 'NOTA_MINIMA_CH',
       'NOTA_MINIMA_CN', 'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA',
       'CPF', 'INSCRICAO_ENEM', 'INSCRITO', 'SEXO', 'DATA_NASCIMENTO',
       'UF_CANDIDATO', 'MUNICIPIO_CANDIDATO', 'OPCAO', 'NOTA_L', 'NOTA_CH',
       'NOTA_CN', 'NOTA_M', 'NOTA_R', 'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO',
       'NOTA_CN_COM_PESO', 'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO',
       'NOTA_CANDIDATO', 'NOTA_CORTE', 'CLASSIFICACAO', 'APROVADO',
       'MATRICULA'],
      dtype='object')
Index(['ANO', 'EDICAO', 'ETAPA', 'DS_ETAPA', 'CODIGO_IES', 'NOME_IES',
       'SIGLA_IES', 'UF_IES', 'CODIGO_CAMPUS', 'NOME_CAMPUS', 'U

In [7]:
# verificar se ainda há colunas com nomes diferentes
columns_df1 = set(df_alvo.columns)
columns_df2 = set(df_ref.columns)

columns_only_in_df1 = sorted(list(columns_df1 - columns_df2))

columns_only_in_df2 = sorted(list(columns_df2 - columns_df1))

max_len = max(len(columns_only_in_df1), len(columns_only_in_df2))
data = {
    "colunas em df_alvo apenas": columns_only_in_df1
    + [""] * (max_len - len(columns_only_in_df1)),
    "colunas em df_ref apenas": columns_only_in_df2
    + [""] * (max_len - len(columns_only_in_df2)),
}
df_differences = pd.DataFrame(data)

print(df_alvo.columns)
print(df_ref.columns)
df_differences

Index(['ANO', 'EDICAO', 'ETAPA', 'DS_ETAPA', 'CODIGO_IES', 'NOME_IES',
       'SIGLA_IES', 'UF_IES', 'CODIGO_CAMPUS', 'NOME_CAMPUS', 'UF_CAMPUS',
       'MUNICIPIO_CAMPUS', 'CODIGO_CURSO', 'NOME_CURSO', 'GRAU', 'TURNO',
       'TIPO_MOD_CONCORRENCIA', 'MOD_CONCORRENCIA', 'PESO_L', 'PESO_CH',
       'PESO_CN', 'PESO_M', 'PESO_R', 'NOTA_MINIMA_L', 'NOTA_MINIMA_CH',
       'NOTA_MINIMA_CN', 'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA',
       'CPF', 'INSCRICAO_ENEM', 'INSCRITO', 'SEXO', 'DATA_NASCIMENTO',
       'UF_CANDIDATO', 'MUNICIPIO_CANDIDATO', 'OPCAO', 'NOTA_L', 'NOTA_CH',
       'NOTA_CN', 'NOTA_M', 'NOTA_R', 'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO',
       'NOTA_CN_COM_PESO', 'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO',
       'NOTA_CANDIDATO', 'NOTA_CORTE', 'CLASSIFICACAO', 'APROVADO',
       'MATRICULA'],
      dtype='object')
Index(['ANO', 'EDICAO', 'ETAPA', 'DS_ETAPA', 'CODIGO_IES', 'NOME_IES',
       'SIGLA_IES', 'UF_IES', 'CODIGO_CAMPUS', 'NOME_CAMPUS', 'UF_CAMPUS',
       'MUNIC

,colunas em df_alvo apenas,colunas em df_ref apenas


In [8]:

try:
    df_columns = pd.DataFrame(
        {"df_alvo": list(df_alvo.columns), "df_ref": list(df_ref.columns)}
    )
except ValueError as e:
    print("colunas não encontradas", e)
    # df_alvo["MATRICULA"] = ''
    # df_columns = pd.DataFrame(
    #     {"df_alvo": list(df_alvo.columns), "df_ref": list(df_ref.columns)}
    # )
df_columns

,df_alvo,df_ref
0,ANO,ANO
1,EDICAO,EDICAO
2,ETAPA,ETAPA
3,DS_ETAPA,DS_ETAPA
4,CODIGO_IES,CODIGO_IES
5,NOME_IES,NOME_IES
6,SIGLA_IES,SIGLA_IES
7,UF_IES,UF_IES
8,CODIGO_CAMPUS,CODIGO_CAMPUS
9,NOME_CAMPUS,NOME_CAMPUS


In [9]:
df_alvo.to_csv(
    transformado_path,
    sep="|",
    encoding="utf-8",
    index=False,
)